In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/finqa-ds/dev.json
/kaggle/input/finqa-ds/train.json
/kaggle/input/finqa-ds/test.json
/kaggle/input/finqa-ds/private_test.json


In [2]:
import json
from pathlib import Path

def convert_finqa(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    converted = []
    for item in raw_data:
        # Soru
        question = item["qa"]["question"]
        answer = item["qa"].get("answer", "")

        # Tabloyu string olarak birleştir
        table = item.get("table", [])
        table_text = "\n".join([" | ".join(row) for row in table])

        # Kaydet
        converted.append({
            "context": table_text,
            "question": question,
            "answer": answer
        })

    # JSONL olarak kaydet
    with open(output_path, 'w', encoding='utf-8') as f:
        for row in converted:
            f.write(json.dumps(row, ensure_ascii=False) + '\n')

# Yollar
base_path = "/kaggle/input/finqa-ds"
out_path = "/kaggle/working"

# Dönüştürme işlemleri
convert_finqa(f"{base_path}/train.json", f"{out_path}/train_converted.jsonl")
convert_finqa(f"{base_path}/dev.json", f"{out_path}/dev_converted.jsonl")
convert_finqa(f"{base_path}/test.json", f"{out_path}/test_converted.jsonl")

print("✔️ Dönüştürme tamamlandı.")

✔️ Dönüştürme tamamlandı.


In [3]:
from datasets import load_dataset

data_files = {
    "train": "/kaggle/working/train_converted.jsonl",
    "validation": "/kaggle/working/dev_converted.jsonl"
}

dataset = load_dataset("json", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

2025-08-07 14:43:11.652068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754577792.015297      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754577792.115516      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
max_input_len = 512
max_target_len = 64

In [5]:
def preprocess(example):
    input_text = f"question: {example['question']} context: {example['context']}"
    model_input = tokenizer(
        input_text,
        max_length=max_input_len,
        truncation=True,
        padding="max_length"
    )
    labels = tokenizer(
        example["answer"],
        max_length=max_target_len,
        truncation=True,
        padding="max_length"
    )
    model_input["labels"] = labels["input_ids"]
    return model_input

In [7]:
tokenized_dataset = dataset.map(preprocess)

Map:   0%|          | 0/6251 [00:00<?, ? examples/s]

Map:   0%|          | 0/883 [00:00<?, ? examples/s]

In [15]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./finqa-trained-model",         
    eval_strategy="epoch",             
    learning_rate=2e-4,                          
    per_device_train_batch_size=4,              
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,                         
    predict_with_generate=True,
    fp16=True,                        
    logging_dir="./logs",
    logging_steps=10,  
    report_to="none",
)


In [16]:
from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipykernel_36/360543815.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [17]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.129500,0.141212
2,0.137400,0.137637
3,0.127900,0.137369


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

TrainOutput(global_step=2346, training_loss=0.26005219856796363, metrics={'train_runtime': 2396.3778, 'train_samples_per_second': 7.826, 'train_steps_per_second': 0.979, 'total_flos': 1.2841254630457344e+16, 'train_loss': 0.26005219856796363, 'epoch': 3.0})

In [18]:
model_output_path = "/kaggle/working/finqa-flan-t5-model"

tokenizer.save_pretrained(model_output_path)
model.save_pretrained(model_output_path)

print("✅ Model ve tokenizer kaydedildi:", model_output_path)

✅ Model ve tokenizer kaydedildi: /kaggle/working/finqa-flan-t5-model
